In [1]:
# Cell 1/7
# -*- coding: utf-8 -*-
import os
import random
import numpy as np
import pandas as pd
from datetime import datetime

import torch
import torch.nn as nn
import torch.optim as optim
import torch.backends.cudnn as cudnn
from torch.utils.data import Dataset, DataLoader
from torch.optim.lr_scheduler import CosineAnnealingLR

from sklearn.preprocessing import StandardScaler, label_binarize
from sklearn.metrics import accuracy_score, roc_auc_score

import matplotlib.pyplot as plt  # 如需画 test loss 曲线会用到

# === 可复现：固定到最大程度 ===
SEED = 42
def set_global_seed(seed: int):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    cudnn.deterministic = True    # 固定 cuDNN 算法
    cudnn.benchmark = False       # 禁止“最优算法搜索”，避免非确定性
set_global_seed(SEED)

# Root path to UCRArchive_2018
ROOT = r".....


In [2]:
# Cell 2/7:
def clean_and_pad_timeseries(raw_2d, min_len=8, cap_len=None, pad_value=0.0,
                             per_sample_standardize=True, fixed_len=None):
    """
    Clean time series with tail NaNs, z-score per-sample (optional), and pad/clip.

    Priority of output length:
      1) fixed_len: if not None, output length = fixed_len (force)
      2) cap_len:   if not None, output length = min(max_real_len, cap_len)
      3) otherwise, output length = max_real_len of this input batch
    """
    import numpy as np

    N, T = raw_2d.shape
    rows, keep_idx, real_lens = [], [], []

    for i in range(N):
        row = raw_2d[i]
        valid_vals = row[~np.isnan(row)]
        L = valid_vals.shape[0]
        if L < min_len:
            continue
        if per_sample_standardize:
            mu = valid_vals.mean()
            sigma = valid_vals.std()
            valid_vals = (valid_vals - mu) / (sigma if sigma > 0 else 1.0)
        rows.append(valid_vals); keep_idx.append(i); real_lens.append(L)

    if len(rows) == 0:
        raise ValueError("All samples filtered out. Lower min_len if needed.")

    max_real_len = max(real_lens)
    if fixed_len is not None:
        target_len = int(fixed_len)
    elif cap_len is not None:
        target_len = min(max_real_len, cap_len)
    else:
        target_len = max_real_len

    out = []
    for arr in rows:
        if arr.shape[0] >= target_len:
            arr = arr[:target_len]
        else:
            arr = np.pad(arr, (0, target_len - arr.shape[0]), constant_values=pad_value)
        out.append(arr)

    X = np.stack(out, axis=0).astype("float32")
    return X, np.array(keep_idx, dtype=np.int64), np.array(real_lens, dtype=np.int64)

In [3]:
# Cell 3/7:
class TwoTowerTransformer(nn.Module):
    """
    Two-tower Transformer:
      - Tower 1: time series tokens [B, T, 1] -> embed -> transformer
      - Tower 2: Aout vector [B, F] as a single token -> embed -> transformer
      - Concat tokens -> final transformer -> flatten -> FC for multi-class logits
    """
    def __init__(self, input_dim1, input_dim2,
                 hidden_dim1, hidden_dim2, hidden_dim3,
                 num_heads, num_layers, num_classes,
                 seq_len1, seq_len2):
        super().__init__()
        # To keep it simple we force equal hidden dims and divisibility by nhead
        assert hidden_dim1 == hidden_dim2 == hidden_dim3, "hidden dims must be equal in this version."
        for h in (hidden_dim1, hidden_dim2, hidden_dim3):
            assert h % num_heads == 0, "hidden_dim must be divisible by num_heads"

        self.embedding1 = nn.Linear(input_dim1, hidden_dim1)
        self.embedding2 = nn.Linear(input_dim2, hidden_dim2)
        self.relu = nn.ReLU()

        self.transformer1 = nn.TransformerEncoder(
            nn.TransformerEncoderLayer(d_model=hidden_dim1, nhead=num_heads, batch_first=True),
            num_layers=num_layers)

        self.transformer2 = nn.TransformerEncoder(
            nn.TransformerEncoderLayer(d_model=hidden_dim2, nhead=num_heads, batch_first=True),
            num_layers=num_layers)

        self.final_transformer = nn.TransformerEncoder(
            nn.TransformerEncoderLayer(d_model=hidden_dim3, nhead=num_heads, batch_first=True),
            num_layers=num_layers)

        self.dropout = nn.Dropout(0.3)
        self.seq_len1 = seq_len1
        self.seq_len2 = 1  # treat Aout as a single token
        fc_in = hidden_dim3 * (seq_len1 + self.seq_len2)
        self.fc = nn.Linear(fc_in, num_classes)

    def forward(self, x1, x2):
        # x1: [B, T, input_dim1], x2: [B, F] or [B, 1, F]
        if x1.dim() == 2:
            x1 = x1.unsqueeze(-1)
        x1 = self.relu(self.embedding1(x1))
        x1 = self.transformer1(x1)

        if x2.dim() == 3:
            assert x2.size(1) == 1, "Expect x2 with L2=1 if 3D"
            x2 = x2.squeeze(1)
        x2 = self.relu(self.embedding2(x2))
        x2 = x2.unsqueeze(1)
        x2 = self.transformer2(x2)

        x = torch.cat((x1, x2), dim=1)
        x = self.final_transformer(x)
        x = x.reshape(x.size(0), -1)
        x = self.dropout(x)
        return self.fc(x)


class VisitDataset(Dataset):
    """Simple tensor dataset for (visit time series, aout features, one-hot labels)."""
    def __init__(self, visit_x, aout_x, y):
        self.visit_x = visit_x.astype("float32")
        self.aout_x  = aout_x.astype("float32")
        self.y       = y.astype("float32")
    def __len__(self): return len(self.y)
    def __getitem__(self, idx):
        return self.visit_x[idx], self.aout_x[idx], self.y[idx]


In [4]:
# %%
# Cell 4/8: run_one_dataset (保存 final_for_test.pt + best_train.pt + metrics.json；不画图)
def run_one_dataset(dataset_dir, dataset_name,
                    device='cuda:0',
                    batch_size=8, num_epochs=100, lr=1e-4,
                    cap_len=None, patience=15):
    import os, json, copy
    import numpy as np, pandas as pd
    import torch, torch.nn as nn, torch.optim as optim
    from torch.utils.data import DataLoader, Dataset
    from torch.optim.lr_scheduler import CosineAnnealingLR
    from sklearn.preprocessing import StandardScaler, label_binarize
    from sklearn.metrics import accuracy_score, roc_auc_score

    # ---------- paths ----------
    tsv_train_path = os.path.join(dataset_dir, f"{dataset_name}_TRAIN_cleaned.tsv")
    tsv_test_path  = os.path.join(dataset_dir, f"{dataset_name}_TEST_cleaned.tsv")
    aout_train_csv = os.path.join(dataset_dir, f"{dataset_name}_Aout_train_k2.csv")
    aout_test_csv  = os.path.join(dataset_dir, f"{dataset_name}_Aout_test_k2.csv")
    if not all(os.path.exists(p) for p in [tsv_train_path, tsv_test_path, aout_train_csv, aout_test_csv]):
        print(f"⚠ Skip {dataset_name}, missing files")
        return None

    # ---------- read (OFFICIAL split) ----------
    tsv_tr = pd.read_csv(tsv_train_path, sep="\t", header=None)
    tsv_te = pd.read_csv(tsv_test_path,  sep="\t", header=None)
    csv_tr = pd.read_csv(aout_train_csv, header=None)
    csv_te = pd.read_csv(aout_test_csv,  header=None)

    y_tr_raw = tsv_tr.iloc[:,0].values
    y_te_raw = tsv_te.iloc[:,0].values
    visit_tr_raw = tsv_tr.iloc[:,1:].values.astype("float32")
    visit_te_raw = tsv_te.iloc[:,1:].values.astype("float32")
    aout_tr_raw_all  = csv_tr.iloc[:,1:].values.astype("float32")
    aout_te_raw_all  = csv_te.iloc[:,1:].values.astype("float32")

    # ---------- length from TRAIN ----------
    tmp_train_clean, keep_tr0, _ = clean_and_pad_timeseries(
        visit_tr_raw, min_len=8, cap_len=cap_len, pad_value=0.0,
        per_sample_standardize=True, fixed_len=None
    )
    train_seq_len = tmp_train_clean.shape[1]

    visit_tr_clean, keep_tr, _ = clean_and_pad_timeseries(
        visit_tr_raw, min_len=8, cap_len=cap_len, pad_value=0.0,
        per_sample_standardize=True, fixed_len=train_seq_len
    )
    y_tr = y_tr_raw[keep_tr]
    aout_tr_raw = aout_tr_raw_all[keep_tr]

    visit_te_clean, keep_te, _ = clean_and_pad_timeseries(
        visit_te_raw, min_len=8, cap_len=cap_len, pad_value=0.0,
        per_sample_standardize=True, fixed_len=train_seq_len
    )
    y_te = y_te_raw[keep_te]
    aout_te_raw = aout_te_raw_all[keep_te]

    # ---------- scaler on TRAIN ----------
    scaler = StandardScaler().fit(aout_tr_raw)
    aout_tr = scaler.transform(aout_tr_raw).astype("float32")
    aout_te = scaler.transform(aout_te_raw).astype("float32")

    # ---------- labels ----------
    classes = sorted(np.unique(y_tr))
    Y_tr = label_binarize(y_tr, classes=classes).astype("float32")
    Y_te = label_binarize(y_te, classes=classes).astype("float32")
    num_classes = Y_tr.shape[1]

    # targets & loss
    if num_classes == 1:
        y_tr_tgt = torch.from_numpy(Y_tr)                         # [N,1] float
        y_te_tgt = torch.from_numpy(Y_te)
        criterion = nn.BCEWithLogitsLoss()
    else:
        y_tr_tgt = torch.from_numpy(np.argmax(Y_tr, axis=1).astype(np.int64))  # [N] long
        y_te_tgt = torch.from_numpy(np.argmax(Y_te, axis=1).astype(np.int64))
        criterion = nn.CrossEntropyLoss()

    # ---------- assemble inputs ----------
    visit_tr = visit_tr_clean[:, :, None]  # [N,L,1]
    visit_te = visit_te_clean[:, :, None]
    seq_len1 = visit_tr.shape[1]
    input_dim1 = visit_tr.shape[2]
    input_dim2 = aout_tr.shape[1]

    # ---------- model ----------
    dev = torch.device(device if torch.cuda.is_available() else 'cpu')
    model = TwoTowerTransformer(
        input_dim1, input_dim2,
        hidden_dim1=16, hidden_dim2=16, hidden_dim3=16,
        num_heads=2, num_layers=2,
        num_classes=num_classes,
        seq_len1=seq_len1, seq_len2=1
    ).to(dev)
    if torch.cuda.device_count() > 1 and str(dev).startswith('cuda'):
        model = nn.DataParallel(model)

    # dataset/dataloader
    class VisitDatasetForTrain(Dataset):
        def __init__(self, visit_x, aout_x, tgt_tensor):
            self.visit_x = visit_x.astype("float32")
            self.aout_x  = aout_x.astype("float32")
            self.tgt     = tgt_tensor
        def __len__(self): return len(self.tgt)
        def __getitem__(self, idx):
            return self.visit_x[idx], self.aout_x[idx], self.tgt[idx]

    g = torch.Generator(device="cpu").manual_seed(SEED)
    train_loader = DataLoader(VisitDatasetForTrain(visit_tr, aout_tr, y_tr_tgt),
                              batch_size=batch_size, shuffle=True, num_workers=0, pin_memory=True,
                              worker_init_fn=lambda wid: np.random.seed(SEED + wid),
                              generator=g)
    test_loader  = DataLoader(VisitDatasetForTrain(visit_te, aout_te, y_te_tgt),
                              batch_size=batch_size, shuffle=False, num_workers=0, pin_memory=True)

    # optimizer/scheduler
    optimizer = optim.Adam(model.parameters(), lr=lr)
    scheduler = CosineAnnealingLR(optimizer, T_max=50, eta_min=0.0)

    # early stopping on TRAIN loss
    class EarlyStopping:
        def __init__(self, patience=15, delta=0.0):
            self.patience=patience; self.delta=delta
            self.best=None; self.count=0; self.stop=False
        def step(self, loss):
            if self.best is None: self.best=loss; return False
            if loss > self.best - self.delta:
                self.count += 1
                if self.count >= self.patience: self.stop=True
            else:
                self.best = loss; self.count = 0
            return self.stop
    es = EarlyStopping(patience=patience)

    # 追踪 train-loss 最优权重
    def _sd(m): return m.module.state_dict() if isinstance(m, nn.DataParallel) else m.state_dict()
    best_train_loss = float("inf")
    best_train_state = None

    # ---------- train ----------
    for epoch in range(num_epochs):
        model.train()
        total = 0.0
        for x1,x2,y in train_loader:
            x1 = x1.to(torch.float32).to(dev)
            x2 = x2.to(torch.float32).to(dev)

            optimizer.zero_grad()
            if num_classes==1:
                y = y.to(torch.float32).to(dev)
            else:
                y = y.to(torch.long).to(dev)

            logits = model(x1,x2)
            loss = criterion(logits, y)
            loss.backward(); optimizer.step()
            total += loss.item()

        scheduler.step()
        avg_train_loss = total / max(1, len(train_loader))

        if avg_train_loss < best_train_loss:
            best_train_loss = avg_train_loss
            best_train_state = copy.deepcopy(_sd(model))

        if es.step(avg_train_loss):
            break

    # ---------- TEST（一次性评估） ----------
    model.eval()
    test_total = 0.0
    logits_list, tgt_list = [], []
    with torch.no_grad():
        for x1,x2,y in test_loader:
            x1 = x1.to(torch.float32).to(dev)
            x2 = x2.to(torch.float32).to(dev)
            if num_classes==1:
                y = y.to(torch.float32).to(dev)
            else:
                y = y.to(torch.long).to(dev)

            logits = model(x1,x2)
            tloss = criterion(logits, y)
            test_total += tloss.item()
            logits_list.append(logits.detach().cpu().numpy())
            tgt_list.append(y.detach().cpu().numpy())
    test_loss = test_total / max(1, len(test_loader))

    logits = np.concatenate(logits_list, axis=0)
    tgt    = np.concatenate(tgt_list, axis=0)

    if num_classes==1:
        probs_pos = 1.0/(1.0+np.exp(-logits.ravel()))
        y_true = tgt.ravel().astype(int)
        y_pred = (probs_pos>=0.5).astype(int)
        test_acc = accuracy_score(y_true, y_pred)
        test_auc = roc_auc_score(y_true, probs_pos)
    else:
        ex = np.exp(logits - logits.max(axis=1, keepdims=True))
        probs = ex / ex.sum(axis=1, keepdims=True)
        y_true = tgt.astype(int)
        y_pred = probs.argmax(axis=1)
        test_acc = accuracy_score(y_true, y_pred)
        y_true_1h = np.eye(num_classes, dtype=int)[y_true]
        test_auc = roc_auc_score(y_true_1h, probs, multi_class='ovr')

    n_test = logits.shape[0]
    print(f"[{dataset_name}] TEST loss={test_loss:.4f} | AUC={test_auc:.4f} | ACC={test_acc:.4f} | n={n_test}")

    # ---------- 保存权重与指标 ----------
    ckpt_dir = os.path.join(dataset_dir, "_twotower_logs"); os.makedirs(ckpt_dir, exist_ok=True)

    # (1) 最终用于测试的权重
    final_path = os.path.join(ckpt_dir, f"{dataset_name}_final_for_test.pt")
    torch.save({
        "model": _sd(model),
        "scaler": scaler,
        "classes": classes,
        "seq_len": int(seq_len1),
        "seed": int(SEED),
        "epochs_run": int(epoch+1),
        "hyper": {"hidden":16, "heads":2, "layers":2, "lr":lr, "batch":batch_size}
    }, final_path)

    # (2) 训练期 train-loss 最优的权重
    best_train_path = os.path.join(ckpt_dir, f"{dataset_name}_best_train.pt")
    if best_train_state is not None:
        torch.save({
            "model": best_train_state,
            "scaler": scaler,
            "classes": classes,
            "seq_len": int(seq_len1),
            "seed": int(SEED),
            "best_train_loss": float(best_train_loss),
            "hyper": {"hidden":16, "heads":2, "layers":2, "lr":lr, "batch":batch_size}
        }, best_train_path)

    # (3) 指标 JSON（含权重路径）
    with open(os.path.join(ckpt_dir, f"{dataset_name}_metrics.json"), "w", encoding="utf-8") as f:
        json.dump({
            "dataset": dataset_name,
            "test_loss": float(test_loss),
            "test_auc": float(test_auc),
            "test_acc": float(test_acc),
            "n_samples": int(n_test),
            "epochs_run": int(epoch+1),
            "seed": int(SEED),
            "weights_final": final_path,
            "weights_best_train": best_train_path
        }, f, ensure_ascii=False, indent=2)

    # 统一 4 个返回值（注意顺序）
    return float(test_auc), float(test_acc), float(test_loss), int(n_test)


In [5]:
# Cell 5/7:
def discover_datasets(root):
    """
    Discover dataset subfolders that contain all four required files:
      *_TRAIN_cleaned.tsv, *_TEST_cleaned.tsv, *_Aout_train_k2.csv, *_Aout_test_k2.csv
    Returns: a sorted list of dataset names (folder names).
    """
    names = []
    for name in sorted(os.listdir(root)):
        subdir = os.path.join(root, name)
        if not os.path.isdir(subdir):
            continue
        t_train = os.path.join(subdir, f"{name}_TRAIN_cleaned.tsv")
        t_test  = os.path.join(subdir, f"{name}_TEST_cleaned.tsv")
        a_train = os.path.join(subdir, f"{name}_Aout_train_k2.csv")
        a_test  = os.path.join(subdir, f"{name}_Aout_test_k2.csv")
        if all(os.path.exists(p) for p in [t_train, t_test, a_train, a_test]):
            names.append(name)
    return names


In [6]:
# %%
# Cell 6/8: run all datasets（解包 4 个返回值；汇总里加 test_loss）
def run_all_datasets(root, device='cuda:0',
                     batch_size=8, num_epochs=100, lr=1e-4,
                     cap_len=None, patience=15):
    dataset_names = discover_datasets(root)
    print(f"Found {len(dataset_names)} datasets:", dataset_names)

    rows = []
    for name in dataset_names:
        out = run_one_dataset(
            dataset_dir=os.path.join(root, name),
            dataset_name=name,
            device=device,
            batch_size=batch_size, num_epochs=num_epochs, lr=lr,
            cap_len=cap_len, patience=patience
        )
        if out is None:
            continue
        test_auc, test_acc, test_loss, n_test = out
        rows.append((name, test_auc, test_acc, test_loss, n_test))

    if not rows:
        print("No dataset finished successfully."); return None

    df = pd.DataFrame(rows, columns=["dataset","test_auc","test_acc","test_loss","n_samples"]).sort_values("dataset")
    mean_auc  = df["test_auc"].mean()
    mean_acc  = df["test_acc"].mean()
    mean_loss = df["test_loss"].mean()
    w_auc = (df["test_auc"] * df["n_samples"]).sum() / df["n_samples"].sum()
    w_acc = (df["test_acc"] * df["n_samples"]).sum() / df["n_samples"].sum()

    print("\n========== Summary (OFFICIAL TEST) ==========")
    print(df.to_string(index=False))
    print(f"\nSimple mean:  AUC = {mean_auc:.4f}, ACC = {mean_acc:.4f}, LOSS = {mean_loss:.4f}")
    print(f"Weighted (by samples): AUC = {w_auc:.4f}, ACC = {w_acc:.4f}")

    summary_dir = os.path.join(root, "_twotower_logs"); os.makedirs(summary_dir, exist_ok=True)
    ts = datetime.now().strftime("%Y%m%d_%H%M%S")
    df.to_csv(os.path.join(summary_dir, f"summary_TEST_{ts}.csv"), index=False, encoding="utf-8")
    with open(os.path.join(summary_dir, f"summary_TEST_{ts}.txt"), "w", encoding="utf-8") as f:
        f.write(df.to_string(index=False))
        f.write(f"\n\nSimple mean:  AUC={mean_auc:.6f}, ACC={mean_acc:.6f}, LOSS={mean_loss:.6f}\n")
        f.write(f"Weighted (by samples): AUC={w_auc:.6f}, ACC={w_acc:.6f}\n")

    return df, (mean_auc, mean_acc, mean_loss), (w_auc, w_acc)


In [7]:
# %%
# Cell 7/8: run ONLY the 6 selected datasets（解包 4 个返回值）
SELECTED = ["ECG200", "GunPoint", "ArrowHead", "Beef", "Coffee", "ECG5000"]

rows = []
for ds in SELECTED:
    ds_dir = os.path.join(ROOT, ds)
    out = run_one_dataset(
        dataset_dir=ds_dir,
        dataset_name=ds,
        device='cuda:0',
        batch_size=8,
        num_epochs=100,
        lr=1e-4,
        cap_len=None,
        patience=15,
    )
    if out is None:
        print(f"[WARN] {ds} 缺少必要文件"); continue
    test_auc, test_acc, test_loss, test_n = out
    rows.append((ds, float(test_auc), float(test_acc), float(test_loss), int(test_n)))
    print(f"[{ds}] AUC={test_auc:.4f} | ACC={test_acc:.4f} | LOSS={test_loss:.4f} | n={test_n}")

# 汇总（六个数据集）
if rows:
    df = pd.DataFrame(rows, columns=["dataset","test_auc","test_acc","test_loss","n_samples"]).sort_values("dataset")
    mean_auc = df["test_auc"].mean(); mean_acc = df["test_acc"].mean(); mean_loss = df["test_loss"].mean()
    w_auc = (df["test_auc"] * df["n_samples"]).sum() / df["n_samples"].sum()
    w_acc = (df["test_acc"] * df["n_samples"]).sum() / df["n_samples"].sum()

    print("\n========== Summary (6 SELECTED | OFFICIAL TEST) ==========")
    print(df.to_string(index=False))
    print(f"\nSimple mean:  AUC = {mean_auc:.4f}, ACC = {mean_acc:.4f}, LOSS = {mean_loss:.4f}")
    print(f"Weighted (by samples): AUC = {w_auc:.4f}, ACC = {w_acc:.4f}")
else:
    print("No dataset finished successfully.")



[ECG200] TEST loss=0.4102 | AUC=0.8950 | ACC=0.8300 | n=100
[ECG200] AUC=0.8950 | ACC=0.8300 | LOSS=0.4102 | n=100
[GunPoint] TEST loss=0.4334 | AUC=0.8718 | ACC=0.7467 | n=150
[GunPoint] AUC=0.8718 | ACC=0.7467 | LOSS=0.4334 | n=150
[ArrowHead] TEST loss=0.7773 | AUC=0.8708 | ACC=0.6400 | n=175
[ArrowHead] AUC=0.8708 | ACC=0.6400 | LOSS=0.7773 | n=175
[Beef] TEST loss=0.9264 | AUC=0.9153 | ACC=0.7333 | n=30
[Beef] AUC=0.9153 | ACC=0.7333 | LOSS=0.9264 | n=30
[Coffee] TEST loss=0.2095 | AUC=0.9897 | ACC=0.9286 | n=28
[Coffee] AUC=0.9897 | ACC=0.9286 | LOSS=0.2095 | n=28
[ECG5000] TEST loss=0.2410 | AUC=0.9329 | ACC=0.9367 | n=4500
[ECG5000] AUC=0.9329 | ACC=0.9367 | LOSS=0.2410 | n=4500

========== Summary (6 SELECTED | OFFICIAL TEST) ==========
  dataset  test_auc  test_acc  test_loss  n_samples
ArrowHead  0.870769  0.640000   0.777283        175
     Beef  0.915278  0.733333   0.926377         30
   Coffee  0.989744  0.928571   0.209466         28
   ECG200  0.894965  0.830000   0.41

In [16]:
# %%
# Cell 8/8: optional — run ALL datasets in ROOT
_ = run_all_datasets(
    root=ROOT,
    device='cuda:0',
    batch_size=8,
    num_epochs=100,
    lr=1e-4,
    cap_len=None,
    patience=15,
)


Found 125 datasets: ['ACSF1', 'Adiac', 'AllGestureWiimoteX', 'AllGestureWiimoteY', 'AllGestureWiimoteZ', 'ArrowHead', 'BME', 'Beef', 'BeetleFly', 'BirdChicken', 'CBF', 'Car', 'Chinatown', 'ChlorineConcentration', 'CinCECGTorso', 'Coffee', 'CricketX', 'CricketY', 'CricketZ', 'Crop', 'DiatomSizeReduction', 'DistalPhalanxOutlineAgeGroup', 'DistalPhalanxOutlineCorrect', 'DistalPhalanxTW', 'DodgerLoopDay', 'DodgerLoopGame', 'DodgerLoopWeekend', 'ECG200', 'ECG5000', 'ECGFiveDays', 'EOGHorizontalSignal', 'EOGVerticalSignal', 'Earthquakes', 'EthanolLevel', 'FaceAll', 'FaceFour', 'FacesUCR', 'FiftyWords', 'Fish', 'FordA', 'FordB', 'FreezerRegularTrain', 'FreezerSmallTrain', 'Fungi', 'GestureMidAirD1', 'GestureMidAirD2', 'GestureMidAirD3', 'GesturePebbleZ1', 'GesturePebbleZ2', 'GunPoint', 'GunPointAgeSpan', 'GunPointMaleVersusFemale', 'GunPointOldVersusYoung', 'Ham', 'HandOutlines', 'Haptics', 'Herring', 'HouseTwenty', 'InlineSkate', 'InsectEPGRegularTrain', 'InsectEPGSmallTrain', 'InsectWingbea

KeyboardInterrupt: 